In [1]:
import psutil
import GPUtil
import time

def get_cpu_usage():
    return psutil.cpu_percent(interval=1)

def get_memory_usage():
    return psutil.virtual_memory().percent

def get_gpu_usage():
    gpus = GPUtil.getGPUs()
    if gpus:
        gpu = gpus[0]
        return gpu.load * 100, gpu.memoryUtil * 100
    return None, None

def monitor_power_usage(duration=10):
    cpu_usage = []
    memory_usage = []
    gpu_usage = []
    gpu_memory_usage = []

    start_time = time.time()
    while time.time() - start_time < duration:
        cpu_usage.append(get_cpu_usage())
        memory_usage.append(get_memory_usage())
        
        gpu, gpu_memory = get_gpu_usage()
        if gpu is not None:
            gpu_usage.append(gpu)
            gpu_memory_usage.append(gpu_memory)
        
        time.sleep(1)
    
    return cpu_usage, memory_usage, gpu_usage, gpu_memory_usage

ModuleNotFoundError: No module named 'distutils'

In [3]:
from ..backend.TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)


cpu_usage, memory_usage, gpu_usage, gpu_memory_usage = monitor_power_usage(10)
tts.tts(text='Ciao amico mio come stai, io sinceramente non sono più sicuro di essere vivo.', speaker_wav='../backend/input/it.wav', language='it')



ImportError: attempted relative import with no known parent package

In [ ]:
import matplotlib.pyplot as plt
#plot all metrics
plt.plot(cpu_usage, label='CPU')
plt.plot(memory_usage, label='Memory')
plt.plot(gpu_usage, label='GPU')
plt.plot(gpu_memory_usage, label='GPU Memory')

plt.legend()
plt.show()